In [20]:
#import ee
#ee.Authenticate()

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt

import pandas as pd
from datetime import datetime

import os
import ipywidgets as widgets

import tkinter as tk
from tkinter import filedialog as fd

In [22]:
# CONSTANTES
#filepath_tch = r'G:\OneDrive - Ingenio Azucarero Guabira S.A\AGRO-CITTCA\PROPIEDADES\DATOS_PARA_GENERAR_GEOJSON\RENDIMIENTO AVANCE 071022.xlsx'
#filepath_bloques = r'G:\OneDrive - Ingenio Azucarero Guabira S.A\AGRO-CITTCA\PROPIEDADES\DATOS_PARA_GENERAR_GEOJSON\BLOQUES.xlsx'

In [2]:
# VARIABLES GLOBALES
RUTA_CATASTRO = ''
CATASTRO = None
PROPIEDAD = None
#TCHs = pd.read_excel(filepath_tch)
#BLOQUES = pd.read_excel(filepath_bloques)

COD_PROP = None
NOM_PROP = None
COD_CA = None
AREA = 0
THC = 0
CUADRANTE = 0

FILE_NAME = ''

In [26]:
# FUNCIONES GENERALES

# open file dialog
def open_file_dialogo():
    root = tk.Tk()
    root.attributes('-topmost', True)
    root.iconify()
    file_name = fd.askopenfilename(parent=root, filetypes=[(".shp", ".shp")])
    root.destroy()
    return file_name

# quita los caracteres especiales de un texto enviado
def quitar_caracteres(text):
    text = text.replace('_', ' ')
    text = text.replace('"', '')
    text = text.replace('.', '')
    return text

# dependiendo del año de la fecha de siembra, retorna 'HOJA', 'SOCA' o 'RENOVACION'
def establecer_estado(fecha):
    date = str(fecha)
    try:
        date = datetime.strptime(date, '%Y-%m-%d')
        year = date.year
        if year == 2022:
            return 'HOJA'
        elif year > 2000:
            return 'SOCA'
        else:
            return 'RENOVACION'
    except Exception as ex:
        return 'RENOVACION'
    
# establece el formato de fecha
def establecer_formato_fecha(fecha):
    fecha = str(fecha)
    if fecha == 'None':
        return '0'
    else:
        return fecha.replace('-', '')

# cargar TCH de cañero a datos de propiedad
def find_tch(cod_ca):
    try:
        return 57.5
    except:
        return 0

# cargar Num. De Cuadrante nax de la propiedad
def find_block(cod_prop):
    try:
        return 359
    except:
        return 0

def convertir_para_dos_gestiones(prop, estado):
    propiedad = prop.astype({
        'unidad_01':'int',
        'unidad_03':'int'
    })
    # cambio de nombre de columnas (id, variedad, textura) por conflicto con las nuevas columnas
    propiedad = propiedad.rename(columns={'id':'cod', 'variedad':'var', 'textura':'tex'})
    # crear nuevas columnas, con valor por defecto
    propiedad['NOM_LOTE'] = propiedad['unidad_05']
    propiedad['TEXTURA'] = propiedad['tex']
    propiedad['SUPERFICIE'] = propiedad['area']
    propiedad['VARIEDAD'] = propiedad['var']
    propiedad['FECHA_SIEM'] = propiedad['fs']
    propiedad['CORTE'] = propiedad['soca']
    propiedad['ESTADO'] = ''
    propiedad['RENDIMIENT'] = 0
    propiedad['COD_CA'] = propiedad['unidad_03']
    propiedad['COD_PROP'] = propiedad['unidad_01']
    propiedad['ID'] = propiedad['cod']
    propiedad['CUADRANTE'] = 0
    # segmenta el gdf propiedad [todas_las_filas , ultimas_13_columnas]
    propiedad = propiedad.iloc[:,-13:]
    # estable estado (hoja, soca, renov)
    propiedad['ESTADO'] = propiedad['FECHA_SIEM'].apply(establecer_estado)
    # cambia el formato de fecha
    propiedad['FECHA_SIEM'] = propiedad['FECHA_SIEM'].apply(establecer_formato_fecha)
    #establecer tipo de variable a nuevas columnas
    propiedad = propiedad.astype({
        'RENDIMIENT':'float',
        'COD_CA':'int',
        'COD_PROP':'int'
    })
    # encontrar thc de cañero
    propiedad['RENDIMIENT'] = propiedad['COD_CA'].apply(find_tch)
    # encontrar cuadrante NAX de propiedad
    propiedad['CUADRANTE']=propiedad['COD_PROP'].apply(find_block)
    # recalcula el ID
    propiedad['ID'] = propiedad.apply(lambda x : '0000' + str(x['COD_PROP']) + '0000' + x['NOM_LOTE'], axis=1)
    # quitar caracteres especiales de la columna variedad
    propiedad['VARIEDAD'] = propiedad['VARIEDAD'].apply(quitar_caracteres)
    # cambia sistema de proyeccion de UTM a WGS84
    propiedad = propiedad.to_crs(epsg=4326)
    # ruta para guardar archivo
    os.chdir(r"G:\OneDrive - Ingenio Azucarero Guabira S.A\AGRO-CITTCA\PROPIEDADES\DATOS_PARA_GENERAR_GEOJSON\PROPIEDADES_OFFICIALES")
    # estructuracion de la ruta
    out_dir = os.path.normpath((os.getcwd() + '/' + FILE_NAME + '_' + str(estado) + '.geojson' ))
    # guardar archivo
    propiedad.to_file(out_dir, driver='GeoJSON')
    print('dddddddddddddddddddddddddddd')
    return propiedad

# convertir propiedad a geojson
def convertir_geojson():    
    # establecer tipo de dato de codigos
    prop_actual = PROPIEDAD[(PROPIEDAD['cultivo']=='canha') | (PROPIEDAD['cultivo']=='siembra')]
    prop_siguiente = PROPIEDAD[PROPIEDAD['cultivo']=='renovacion']
    
    propiedad = convertir_para_dos_gestiones(PROPIEDAD, 2025)
    #propiedad = convertir_para_dos_gestiones(prop_siguiente, 2026)
    
    out_lotes.clear_output()
    with out_lotes:
        display(propiedad)

In [27]:
# FUNNCIONES DE WIDGETS

# btn_cargar_catastro
def cargar_catastro(gui):
    global CATASTRO
    file_name = open_file_dialogo()
    txt_path_catastro.value = file_name
    CATASTRO = gpd.read_file(file_name)

def cargar_propiedad(gui):
    global PROPIEDAD, COD_PROP, NOM_PROP, COD_CA, AREA, TCH, CUADRANTE, FILE_NAME
    # selecciona la propiedad segun el codigo prop
    PROPIEDAD = CATASTRO[CATASTRO['unidad_01']==txt_cod_prop.value]
    
    # extraer datos de propiedad
    COD_PROP = PROPIEDAD.iloc[0]['unidad_01']
    NOM_PROP = PROPIEDAD.iloc[0]['unidad_02']
    COD_CA = PROPIEDAD.iloc[0]['unidad_03']
    
    # total area
    AREA = PROPIEDAD['area'].sum()
    # encuentra el TCH promedio de cañero, en caso de encontrar retorna 0
    TCH = find_tch(COD_CA)
    # encuentra el cuadrnate NAX de la propiedad, en caso de no encontrar retorna 0
    CUADRANTE = find_block(COD_PROP)
    
    FILE_NAME = str(COD_PROP) + ' - ' + quitar_caracteres(NOM_PROP)
    
    info = '''
        PROPIEDAD: {file_name} 
        AREA: {area}
        TCH: {tch}
        CUADRANTE: {cuadrante}
        '''.format(file_name=FILE_NAME, area=AREA, tch=TCH, cuadrante=CUADRANTE)
    
    # output: codigo y nombre de propiedad
    out_nom_prop.clear_output()
    with out_nom_prop:
        print(info)
    
    # output: PLANO
    out_prop.clear_output()
    with out_prop:
        PROPIEDAD.plot()
        plt.show()   
        
    out_lotes.clear_output()
    with out_lotes:
        display(PROPIEDAD)

def convertir_a_geojson(gui):
    convertir_geojson()

In [28]:
# textbox ruta de catastro
txt_path_catastro = widgets.Text(
    value=RUTA_CATASTRO,
    disabled=False,
    #layout=widgets.Layout(width='auto', height='40px')
)

# boton cargar ruta de catastro
btn_cargar_catastro = widgets.Button(
    description='Cargar Catastro',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cargar',
)
btn_cargar_catastro.on_click(cargar_catastro)

# textbox codigo propiedad
txt_cod_prop = widgets.BoundedIntText(
    value=30,
    min=0,
    max=99999999999999999,
    step=1,
    disabled=False
)

# boton cargar propiedad
btn_cargar_propiedad = widgets.Button(
    description='Cargar Propiedad',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cargar propiedad',
)
btn_cargar_propiedad.on_click(cargar_propiedad)

# boton convertir a geojson
btn_convertir_a_geojson = widgets.Button(
    description='Convertir Propiedad',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Convertir a GEOJSON',
)
btn_convertir_a_geojson.on_click(convertir_a_geojson)

In [29]:
form_item_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    width='100%'
)

form_items = [
    widgets.Box([widgets.Label(value='CATASTRO:'), txt_path_catastro], layout=form_item_layout),
    widgets.Box([btn_cargar_catastro], layout=form_item_layout),
    widgets.Box([widgets.Label(value='COD PROP:'), txt_cod_prop], layout=form_item_layout),
    widgets.Box([btn_cargar_propiedad], layout=form_item_layout),
    widgets.Box([btn_convertir_a_geojson], layout=form_item_layout)
]

form = widgets.Box(form_items, layout=widgets.Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='100%'
))
form

Box(children=(Box(children=(Label(value='CATASTRO:'), Text(value='')), layout=Layout(display='flex', flex_flow…

### DATOS

In [30]:
out_nom_prop = widgets.Output(layout={'border': '1px solid black'})
out_nom_prop

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### PROPIEDAD

In [31]:
out_prop = widgets.Output(layout={'border': '1px solid black'})
out_prop

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

## LOTES

In [32]:
out_lotes = widgets.Output(layout={'border': '1px solid black'})
out_lotes

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…